# Tree Methods

## Imported Libraries

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib
import csv

## Data Extraction

In [2]:
trainingData = pd.read_csv("../Data/train.csv")
trainingData.hist(bins = 10)

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x10201FB0>,
      dtype=object)

## Binning Analog Data

In [3]:
AgeBins = pd.IntervalIndex.from_tuples([(0,15),(15,30),(30,45),(45,60),(60,75),(75,90),(90,105),(105,120)])
TicketBins = pd.IntervalIndex.from_tuples([(0,50000), (50000,100000), (100000,150000),(150000,200000),(200000, 250000), (250000, 300000), (300000, 350000)])
CostBins = pd.IntervalIndex.from_tuples([(-1, 50),(50,100), (100,150), (150, 200), (200, 250), (250, 300), (300,350), (350,400), (400, 450), (450, 500), (500, 550)])

trainingData["AgeGroup"] = pd.cut(trainingData['Age'], bins=AgeBins).cat.add_categories(pd.Interval(-2,-1)).fillna(pd.Interval(-2,-1))
trainingData['TicketGroup'] = pd.cut(pd.to_numeric(trainingData['Ticket'], errors="coerce"), bins=TicketBins).cat.add_categories(pd.Interval(-2,-1)).fillna(pd.Interval(-2,-1))
trainingData['CostGroup'] = pd.cut(trainingData['Fare'], bins=CostBins).cat.add_categories(pd.Interval(-2,-1)).fillna(pd.Interval(-2,-1))
trainingData['Embarked'] = trainingData['Embarked'].fillna("S")

## CV Groups

In [4]:
cvTrain1 = trainingData[0:(4*trainingData["Survived"].count()//5)]
cvTest1 = trainingData[(4*trainingData["Survived"].count()//5):]

cvTrain2 = trainingData[0:(3*trainingData["Survived"].count()//5)].append(trainingData[(4*trainingData["Survived"].count()//5):])
cvTest2 = trainingData[(3*trainingData["Survived"].count()//5):(4*trainingData["Survived"].count()//5)]

cvTrain3 = trainingData[0:(2*trainingData["Survived"].count()//5)].append(trainingData[(3*trainingData["Survived"].count()//5):])
cvTest3 = trainingData[(2*trainingData["Survived"].count()//5):(3*trainingData["Survived"].count()//5)]

cvTrain4 = trainingData[0:trainingData["Survived"].count()//5].append(trainingData[(2*trainingData["Survived"].count()//5):])
cvTest4 = trainingData[trainingData["Survived"].count()//5:(2*trainingData["Survived"].count()//5)]

cvTrain5 = trainingData[trainingData["Survived"].count()//5:]
cvTest5 = trainingData[0:trainingData["Survived"].count()//5]

## Tree Data Structure

In [31]:
class Tree:
    
    def __init__(self, feature, children):
        self.feature = feature
        self.children = children
        
    def getFeature(self):
        return self.feature
    
    def search(self,datapoint):
        if self.children is None:
            return self.getFeature()
        point = datapoint.get(self.feature)
        for child in self.children:
            if child[0] == point:
                return child[1].search(datapoint)
        

## Tree Structure Test

In [6]:
leaf1 = Tree(0, None)
leaf2 = Tree(1, None)
leaf3 = Tree(1, None)

branch = Tree("test2", [["b1",leaf1], ["b2",leaf2]])
root = Tree("test1", [["a1",branch], ["a2",leaf3]])

EXdata = pd.Series(data=["a1","b1"],index=["test1","test2"])

assert 0 == root.search(EXdata)

## Information Gain Functions

In [7]:
def Entropy(dataSet):
    result1 = dataSet[dataSet['Survived'] == 1].size/dataSet.size
    if result1 == 0 or result1 == 1:
        return 0
    result1 = result1*np.log2(result1)
    result2 = dataSet[dataSet['Survived'] == 0].size/dataSet.size
    result2 =  result2*np.log2(result2)
    return -1* (result1 + result2)

In [8]:
def InformationGain(dataSet, feature):
    
    totalE = Entropy(dataSet)
    sumE = 0
    
    values, counts = np.unique(dataSet[feature], return_counts = True)
    for val in values:
        sumE += (dataSet[dataSet[feature] == val].size)/dataSet.size*Entropy(dataSet[dataSet[feature] == val])
    return totalE - sumE
        

## Tree Builder

In [9]:
def BuildTree(depth,features,dataset):
    
    values, counts = np.unique(dataset["Survived"], return_counts = True)
    if counts[0] == dataset.size or depth == 0:
        return Tree(values[0], None)
    
        
    
    best = None
    bestIG = 0
    for feat in features:
        IG = InformationGain(dataset, feat)
        if best == None or bestIG < IG:
            best = feat
            bestIG = IG
    
    
    children = []
    if depth == 1 or len(features) == 1:
        for val in np.unique(dataset[best]):
            children.append([val,Tree(dataset[dataset[best] == val].mode().loc[0, "Survived"], None)])
    else:
        subset = features.copy()
        subset.remove(best)
        for val in np.unique(dataset[best]):
            
            children.append([val, BuildTree(depth-1, subset, dataset[dataset[best] == val])])
                            
    return Tree(best, children)



## Testing Tree Builder

In [10]:
testTree = BuildTree(3, ["AgeGroup", "CostGroup", "TicketGroup"], trainingData)
testTree.search(trainingData.loc[4])

0.0

## Accuracy

In [11]:
def accuracy(dataSet, tree):
    
    correct = 0
    for ID in dataSet["PassengerId"]:
        dataPoint = dataSet.loc[ID-1]
        if dataPoint["Survived"] == tree.search(dataPoint):
            correct+=1
        
    return correct/dataSet["PassengerId"].count()

## Hyper Parameter Testing

In [12]:
depths = [1,2,3,4,5]
feats = ["Pclass", "Sex", "AgeGroup", "SibSp", "Parch","TicketGroup", "CostGroup", "Embarked"]
results = {}

for depth in depths:
    accuracies = []
    
    tree = BuildTree(depth,feats,cvTrain1)
    accuracies.append(accuracy(cvTest1, tree))
    
    tree = BuildTree(depth,feats,cvTrain2)
    accuracies.append(accuracy(cvTest2, tree))
    
    tree = BuildTree(depth,feats,cvTrain3)
    accuracies.append(accuracy(cvTest3, tree))
    
    tree = BuildTree(depth,feats,cvTrain4)
    accuracies.append(accuracy(cvTest4, tree))
    
    tree = BuildTree(depth,feats,cvTrain5)
    accuracies.append(accuracy(cvTest5, tree))
    
    results[depth] = accuracies

In [13]:
for result in results:
    print("Depth: " + str(result))
    print(str(sum(results[result])/len(results[result])))
    print()

Depth: 1
0.7867428284476806

Depth: 2
0.768765300357793

Depth: 3
0.8001506496767309

Depth: 4
0.7934341849224782

Depth: 5
0.7317054798819911



## Self-Accuracy Test

In [32]:
learnedTree = BuildTree(3,feats,trainingData)

print("Training Set Accuracy: ", accuracy(trainingData,learnedTree))

Training Set Accuracy:  0.8327721661054994


## Processing Test Set

In [29]:
testData = pd.read_csv("../Data/train.csv")

testData["AgeGroup"] = pd.cut(testData['Age'], bins=AgeBins).cat.add_categories(pd.Interval(-2,-1)).fillna(pd.Interval(-2,-1))
testData['TicketGroup'] = pd.cut(pd.to_numeric(testData['Ticket'], errors="coerce"), bins=TicketBins).cat.add_categories(pd.Interval(-2,-1)).fillna(pd.Interval(-2,-1))
testData['CostGroup'] = pd.cut(testData['Fare'], bins=CostBins).cat.add_categories(pd.Interval(-2,-1)).fillna(pd.Interval(-2,-1))
testData['Embarked'] = testData['Embarked'].fillna("S")



## Analyzing Test Set

In [35]:
with open('ID3_Disc_Tree.csv', mode='w', newline='') as treedone:
    treedone = csv.writer(treedone)
    treedone.writerow(['PassengerId', 'Survived'])
    
    for index in range(418):
        label = learnedTree.search(testData.loc[index])
        print("Passenger: " + str(index) + ", Label: " + str(label))
        if label == 0:
            treedone.writerow([index+892, 0])
        else:
            treedone.writerow([index+892, 1])

Passenger: 0, Label: 0.0
Passenger: 1, Label: 1.0
Passenger: 2, Label: 0.0
Passenger: 3, Label: 1.0
Passenger: 4, Label: 0.0
Passenger: 5, Label: 0.0
Passenger: 6, Label: 0.0
Passenger: 7, Label: 0.0
Passenger: 8, Label: 0.0
Passenger: 9, Label: 1.0
Passenger: 10, Label: 0.0
Passenger: 11, Label: 1.0
Passenger: 12, Label: 0.0
Passenger: 13, Label: 0.0
Passenger: 14, Label: 0.0
Passenger: 15, Label: 1.0
Passenger: 16, Label: 0.0
Passenger: 17, Label: 0.0
Passenger: 18, Label: 0.0
Passenger: 19, Label: 1.0
Passenger: 20, Label: 0.0
Passenger: 21, Label: 0.0
Passenger: 22, Label: 1.0
Passenger: 23, Label: 0.0
Passenger: 24, Label: 0.0
Passenger: 25, Label: 0.0
Passenger: 26, Label: 0.0
Passenger: 27, Label: 0.0
Passenger: 28, Label: 1.0
Passenger: 29, Label: 0.0
Passenger: 30, Label: 0.0
Passenger: 31, Label: 1.0
Passenger: 32, Label: 1.0
Passenger: 33, Label: 0.0
Passenger: 34, Label: 0.0
Passenger: 35, Label: 0.0
Passenger: 36, Label: 0.0
Passenger: 37, Label: 0.0
Passenger: 38, Label: 

Passenger: 367, Label: 1.0
Passenger: 368, Label: 1.0
Passenger: 369, Label: 1.0
Passenger: 370, Label: 0.0
Passenger: 371, Label: 0.0
Passenger: 372, Label: 0.0
Passenger: 373, Label: 0.0
Passenger: 374, Label: 0.0
Passenger: 375, Label: 1.0
Passenger: 376, Label: 0.0
Passenger: 377, Label: 0.0
Passenger: 378, Label: 0.0
Passenger: 379, Label: 0.0
Passenger: 380, Label: 1.0
Passenger: 381, Label: 1.0
Passenger: 382, Label: 0.0
Passenger: 383, Label: 1.0
Passenger: 384, Label: 0.0
Passenger: 385, Label: 0.0
Passenger: 386, Label: 0.0
Passenger: 387, Label: 1.0
Passenger: 388, Label: 0.0
Passenger: 389, Label: 1.0
Passenger: 390, Label: 0.0
Passenger: 391, Label: 0.0
Passenger: 392, Label: 0.0
Passenger: 393, Label: 1.0
Passenger: 394, Label: 0.0
Passenger: 395, Label: 0.0
Passenger: 396, Label: 0.0
Passenger: 397, Label: 0.0
Passenger: 398, Label: 0.0
Passenger: 399, Label: 1.0
Passenger: 400, Label: 0.0
Passenger: 401, Label: 0.0
Passenger: 402, Label: 0.0
Passenger: 403, Label: 0.0
P